In [1]:
import datetime
import json
import os
import uuid

import requests
from authlib.jose import jwt

from fhirpy import SyncFHIRClient

In [2]:
BASE_URL = "https://fhir.epic.com/interconnect-fhir-oauth"

In [3]:
# Token expiration time should not be more then 5 minutes since the current
exp = int((datetime.datetime.now() + datetime.timedelta(minutes=4)).timestamp())

# https://fhir.epic.com/Documentation?docId=oauth2&section=Creating-JWTs
jwt_header = {"alg": "RS384", "typ": "JWT"}
payload = {
    "iss": "", # non-production ID, see screnshort below
    "sub": "", # non-production ID, see screnshort below
    "aud": f"{BASE_URL}/oauth2/token",
    "jti": str(uuid.uuid4()), # should be uniq for every token request within the exp 
    "exp": exp,
}

In [4]:
# https://fhir.epic.com/Documentation?docId=oauth2&section=Creating-Key-Pair_OpenSSL
with open("privatekey.pem") as f: # path to generated private .pem file
    private_key = f.read()

In [5]:
# You can read the file with private key
# or use its content as a string variable for testing purposes
encoded_token = jwt.encode(jwt_header, payload, private_key)

In [7]:
# https://fhir.epic.com/Documentation?docId=oauth2&section=Backend-Oauth2_Access-Token-Request
data = {
    "grant_type": "client_credentials",
    "client_id": "6079e18c-b8d7-48a8-b87a-e5a74b68e0af",
    "client_assertion_type": "urn:ietf:params:oauth:client-assertion-type:jwt-bearer",
    "client_assertion": encoded_token
}
headers = requests.structures.CaseInsensitiveDict(
    [("Content-Type", "application/x-www-form-urlencoded")]
)

In [8]:
response = requests.post(
    f"{BASE_URL}/oauth2/token",
    headers=headers,
    data=data,
)
print(json.dumps(response.json(), indent=4, separators=(',', ': ')))

{
    "error": "invalid_client",
    "error_description": null
}


In [9]:
access_token = response.json()["access_token"]

KeyError: 'access_token'

#### init FHIRClient

In [ ]:
FHIR_BASE_URL = f"{BASE_URL}/api/FHIR/R4"

In [ ]:
client = SyncFHIRClient(
    FHIR_BASE_URL,
    authorization=f"Bearer {access_token}",
)
client

#### Get Patient

In [ ]:
# https://fhir.epic.com/Sandbox?api=931
patient = client.reference("Patient", "eJzlzKe3KPzAV5TtkxmNivQ3").to_resource().serialize()

print(json.dumps(patient, indent=2, separators=(',', ': ')))